# Question Generator - Implantação

Utiliza um transformer T5 pré treinado em português e disponibilizado pelo [huggingfaces](https://platiagro.github.io/tutorials/).<br>

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
from expander import DocExpander
from aux_functions import build_df_result


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/qgenerator.joblib")
        self.model = artifacts["model"]
        self.expand_context = artifacts["expand_context"]
        self.infer_num_gen_sentences = artifacts["infer_num_gen_sentences"]
        self.column_context = artifacts["column_context"]
        self.column_question = artifacts["column_question"]

    def expand(self,df):
        if self.expand_context:
            exp = DocExpander() 
            df_final = exp.expand_sql(df,context_column_name=self.column_context,questions_column_name = self.column_question)
        
        return df_final

    def predict(self, X, feature_names, meta=None):
        feature_names_pipeline = ['doc_id', 'context']
        feature_names_qa = ['context']
        
        if feature_names != feature_names_pipeline and feature_names != feature_names_qa:
            raise ValueError(f'feature_names deve ser {feature_names_pipeline} ou {feature_names_qa}')
        
        
        df_input = pd.DataFrame(X,columns=feature_names)
        contexts = df_input['context'].to_numpy()
        gen_questions_dict = self.model.forward(contexts=contexts, num_gen_sentences=self.infer_num_gen_sentences)
        df_result = build_df_result(gen_questions_dict,column_context=self.column_context,column_question=self.column_question)
        df_result = self.expand(df_result)
        
        if feature_names == feature_names_pipeline:
            df_input = df_input[['doc_id']] 
            df_input['index'] = df_input.index
            df_result['index'] = df_result.index
            df_result = pd.merge(df_input, df_result, on='index', how='outer')
            del df_result['index']
            
        #return df.to_numpy()
        return df_result

Overwriting Model.py


In [2]:
import pandas as pd
df = pd.read_csv("squad-test-v1.1.csv")
n_lines = 10
contexts = df['context'][:n_lines]
indexes = df.index[:n_lines]

df_small = pd.DataFrame({'doc_id':indexes,'context':contexts})
X = df_small.to_numpy()

In [3]:
from Model import Model
model = Model()
result = model.predict(X,['doc_id','context'])
result

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 2/2 [00:06<00:00,  3.42s/it]


,doc_id,context,questions,expanded_context
0,0,Doctor Who é um programa de televisão de ficçã...,"[Que tipo de nave é Doctor Who?, O que é uma c...",Doctor Who é um programa de televisão de ficçã...
1,1,"O Newcastle Beer Festival, organizado pela CAM...",[Em que mês acontece o Newcastle Beer Festival...,"O Newcastle Beer Festival, organizado pela CAM..."
2,2,"Por outro lado, Lutero também aponta que os De...","[Qual é o nome de um termo para o que?, O que ...","Por outro lado, Lutero também aponta que os De..."
3,3,A informática em farmácia é a combinação da ci...,"[O que é informática em farmácia?, Quando os i...",A informática em farmácia é a combinação da ci...
4,4,O Super Bowl 50 apresentou inúmeros registros ...,"[Em que ano a NBA venceu o Super Bowl 50?, Qua...",O Super Bowl 50 apresentou inúmeros registros ...
5,5,O outro terço da água flui através do Pannerde...,"[Quando a água flui no Pannerdens Kanaal?, Que...",O outro terço da água flui através do Pannerde...
6,6,Um problema é considerado intrinsecamente difí...,[O que é considerado intrinsecamente difícil q...,Um problema é considerado intrinsecamente difí...
7,7,"Nos anos seguintes, Hoelun e seus filhos viver...","[Quando o Temujin se tornou adulto?, Qual é o ...","Nos anos seguintes, Hoelun e seus filhos viver..."
8,8,Os anos 70 foram destacados por várias séries ...,[Qual é o nome do filme que apresentou a mulhe...,Os anos 70 foram destacados por várias séries ...
9,9,"As práticas, tecnologias e recursos de constru...","[O que significa a construção residencial?, Qu...","As práticas, tecnologias e recursos de constru..."


In [4]:
import torch 
import gc

gc.collect()
torch.cuda.empty_cache()

In [5]:
del model